# Challenge 2: Sentiment Analysis

In this challenge we will learn sentiment analysis and practice performing sentiment analysis on Twitter tweets. 

## Introduction

Sentiment analysis is to *systematically identify, extract, quantify, and study affective states and subjective information* based on texts ([reference](https://en.wikipedia.org/wiki/Sentiment_analysis)). In simple words, it's to understand whether a person is happy or unhappy in producing the piece of text. Why we (or rather, companies) care about sentiment in texts? It's because by understanding the sentiments in texts, we will be able to know if our customers are happy or unhappy about our products and services. If they are unhappy, the subsequent action is to figure out what have caused the unhappiness and make improvements.

Basic sentiment analysis only understands the *positive* or *negative* (sometimes *neutral* too) polarities of the sentiment. More advanced sentiment analysis will also consider dimensions such as agreement, subjectivity, confidence, irony, and so on. In this challenge we will conduct the basic positive vs negative sentiment analysis based on real Twitter tweets.

NLTK comes with a [sentiment analysis package](https://www.nltk.org/api/nltk.sentiment.html). This package is great for dummies to perform sentiment analysis because it requires only the textual data to make predictions. For example:

```python
>>> from nltk.sentiment.vader import SentimentIntensityAnalyzer
>>> txt = "Ironhack is a Global Tech School ranked num 2 worldwide.   Our mission is to help people transform their careers and join a thriving community of tech professionals that love what they do."
>>> analyzer = SentimentIntensityAnalyzer()
>>> analyzer.polarity_scores(txt)
{'neg': 0.0, 'neu': 0.741, 'pos': 0.259, 'compound': 0.8442}
```

In this challenge, however, you will not use NLTK's sentiment analysis package because in your Machine Learning training in the past 2 weeks you have learned how to make predictions more accurate than that. The [tweets data](https://www.kaggle.com/kazanova/sentiment140) we will be using today are already coded for the positive/negative sentiment. You will be able to use the Naïve Bayes classifier you learned in the lesson to predict the sentiment of tweets based on the labels.

## Conducting Sentiment Analysis

### Loading and Exploring Data

The dataset we'll be using today is located in the lab directory named `Sentiment140.csv.zip`. You need to unzip it into a `.csv` file. Then in the cell below, load and explore the data.

*Notes:* 

* The dataset was downloaded from [Kaggle](https://www.kaggle.com/kazanova/sentiment140). We made a slight change on the original data so that each column has a label.

* The dataset is huuuuge (1.6m tweets). When you develop your data analysis codes, you can sample a subset of the data (e.g. 20k records) so that you will save a lot of time when you test your codes.

In [1]:
# your code here

import pandas as pd

dataset = pd.read_csv('/Users/manuelaquino/Bootcamp/dataptmad1120/module-3/natural-language-processing/data/training.1600000.processed.noemoticon.csv')
print(dataset.shape)
dataset.head()

(1599999, 6)


,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, that's a bummer. You shoulda got David Carr of Third Day to do it. ;D"
0,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
1,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
2,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
3,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."
4,0,1467811372,Mon Apr 06 22:20:00 PDT 2009,NO_QUERY,joy_wolf,@Kwesidei not the whole crew


In [2]:
dataset_sample = dataset.sample(n=20000)
dataset_sample.shape

(20000, 6)

In [3]:
dataset_sample.to_csv('/Users/manuelaquino/Bootcamp/dataptmad1120/module-3/natural-language-processing/data/dataset_sample.csv')

In [4]:
dataset_sample

,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, that's a bummer. You shoulda got David Carr of Third Day to do it. ;D"
972491,4,1832580452,Sun May 17 21:03:18 PDT 2009,NO_QUERY,TMinor91,"@starbucks For the first time today, had a ski..."
816744,4,1551524542,Sat Apr 18 09:17:05 PDT 2009,NO_QUERY,netfairy23,"Prolly goin out 2nite, yay! Gonna see No More ..."
83615,0,1753392676,Sun May 10 01:25:01 PDT 2009,NO_QUERY,fryfan20,"I miss the Moderator on the fry forum, non of ..."
870040,4,1678267690,Sat May 02 06:07:59 PDT 2009,NO_QUERY,jhgagle,@Pastor_T meh...it's just a C5
851345,4,1565799716,Mon Apr 20 07:34:46 PDT 2009,NO_QUERY,WLShafor,@kevinrose Tgiers Milk sounds good
...,...,...,...,...,...,...
609565,0,2223766130,Thu Jun 18 08:41:04 PDT 2009,NO_QUERY,lalikesmusic,woke up really crappy hoping today is a bette...
978745,4,1833865856,Mon May 18 00:44:19 PDT 2009,NO_QUERY,DeliciousLunch,"@Col_RFTL Well, we've been discussing Shrews d..."
408695,0,2059463939,Sat Jun 06 16:59:20 PDT 2009,NO_QUERY,andretouch,Gay guys are mean to me or they're just bitch...
40959,0,1573921698,Tue Apr 21 03:29:12 PDT 2009,NO_QUERY,cittiecait,how come i never win anything: #lg15 #harpersg...


In [5]:
columns_names = ['target', 'id', 'date', 'flag', 'user', 'text' ]

In [6]:
dataset_sample.columns = columns_names
dataset_sample

,target,id,date,flag,user,text
972491,4,1832580452,Sun May 17 21:03:18 PDT 2009,NO_QUERY,TMinor91,"@starbucks For the first time today, had a ski..."
816744,4,1551524542,Sat Apr 18 09:17:05 PDT 2009,NO_QUERY,netfairy23,"Prolly goin out 2nite, yay! Gonna see No More ..."
83615,0,1753392676,Sun May 10 01:25:01 PDT 2009,NO_QUERY,fryfan20,"I miss the Moderator on the fry forum, non of ..."
870040,4,1678267690,Sat May 02 06:07:59 PDT 2009,NO_QUERY,jhgagle,@Pastor_T meh...it's just a C5
851345,4,1565799716,Mon Apr 20 07:34:46 PDT 2009,NO_QUERY,WLShafor,@kevinrose Tgiers Milk sounds good
...,...,...,...,...,...,...
609565,0,2223766130,Thu Jun 18 08:41:04 PDT 2009,NO_QUERY,lalikesmusic,woke up really crappy hoping today is a bette...
978745,4,1833865856,Mon May 18 00:44:19 PDT 2009,NO_QUERY,DeliciousLunch,"@Col_RFTL Well, we've been discussing Shrews d..."
408695,0,2059463939,Sat Jun 06 16:59:20 PDT 2009,NO_QUERY,andretouch,Gay guys are mean to me or they're just bitch...
40959,0,1573921698,Tue Apr 21 03:29:12 PDT 2009,NO_QUERY,cittiecait,how come i never win anything: #lg15 #harpersg...


In [7]:
dataset_sample.index

Int64Index([ 972491,  816744,   83615,  870040,  851345,  725762, 1161958,
             776949,  367049,  356841,
            ...
            1133504, 1327834,  346677, 1011101, 1525979,  609565,  978745,
             408695,   40959,  954008],
           dtype='int64', length=20000)

### Prepare Textual Data for Sentiment Analysis

Now, apply the functions you have written in Challenge 1 to your whole data set. These functions include:

* `clean_up()`

* `tokenize()`

* `stem_and_lemmatize()`

* `remove_stopwords()`

Create a new column called `text_processed` in the dataframe to contain the processed data. At the end, your `text_processed` column should contain lists of word tokens that are cleaned up. Your data should look like below:

![Processed Data](data-cleaning-results.png)

In [8]:
import re

def clean_up(s):
    clean = re.sub('[ ]+',' ', re.sub('(http(.*) )|([^a-z ])',' ',s.lower()).lstrip().rstrip())
    return clean


import spacy
import en_core_web_sm

def stem_and_lemmatize(l):
    nlp = spacy.load('en_core_web_sm')
    tokens = nlp(l)
    return str(tokens)

import nltk
nltk.download('punkt')
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize


def remove_stopwords(text_string):
    stop_words = set(stopwords.words('english'))
    word_tokens = word_tokenize(text_string)
    filtered = [w for w in word_tokens if not w in stop_words]
    return filtered

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/manuelaquino/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/manuelaquino/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [9]:
# your code here

def bag_of_words(text):
    a = clean_up(text)
    b = stem_and_lemmatize(a)
    c = remove_stopwords(b)
    return c

In [10]:
bag_of_words("""@Ironhack's-#Q website 776-is http://ironhack.com [(2018)]")""")

['ironhack', 'q', 'website']

### Creating Bag of Words

The purpose of this step is to create a [bag of words](https://en.wikipedia.org/wiki/Bag-of-words_model) from the processed data. The bag of words contains all the unique words in your whole text body (a.k.a. *corpus*) with the number of occurrence of each word. It will allow you to understand which words are the most important features across the whole corpus.

Also, you can imagine you will have a massive set of words. The less important words (i.e. those of very low number of occurrence) do not contribute much to the sentiment. Therefore, you only need to use the most important words to build your feature set in the next step. In our case, we will use the top 5,000 words with the highest frequency to build the features.

In the cell below, combine all the words in `text_processed` and calculate the frequency distribution of all words. A convenient library to calculate the term frequency distribution is NLTK's `FreqDist` class ([documentation](https://www.nltk.org/api/nltk.html#module-nltk.probability)). Then select the top 5,000 words from the frequency distribution.

In [11]:
# your code here

from tqdm import tqdm
tqdm.pandas()

dataset_sample['text_processed'] = dataset_sample['text'].progress_apply(lambda x: bag_of_words(x))

100%|██████████| 20000/20000 [4:22:58<00:00,  1.27it/s]     


In [12]:
dataset_sample

,target,id,date,flag,user,text,text_processed
972491,4,1832580452,Sun May 17 21:03:18 PDT 2009,NO_QUERY,TMinor91,"@starbucks For the first time today, had a ski...","[starbucks, first, time, today, skinny, mocha,..."
816744,4,1551524542,Sat Apr 18 09:17:05 PDT 2009,NO_QUERY,netfairy23,"Prolly goin out 2nite, yay! Gonna see No More ...","[prolly, goin, nite, yay, gon, na, see, goodby..."
83615,0,1753392676,Sun May 10 01:25:01 PDT 2009,NO_QUERY,fryfan20,"I miss the Moderator on the fry forum, non of ...","[miss, moderator, fry, forum, non, seem, come,..."
870040,4,1678267690,Sat May 02 06:07:59 PDT 2009,NO_QUERY,jhgagle,@Pastor_T meh...it's just a C5,"[pastor, meh, c]"
851345,4,1565799716,Mon Apr 20 07:34:46 PDT 2009,NO_QUERY,WLShafor,@kevinrose Tgiers Milk sounds good,"[kevinrose, tgiers, milk, sounds, good]"
...,...,...,...,...,...,...,...
609565,0,2223766130,Thu Jun 18 08:41:04 PDT 2009,NO_QUERY,lalikesmusic,woke up really crappy hoping today is a bette...,"[woke, really, crappy, hoping, today, better, ..."
978745,4,1833865856,Mon May 18 00:44:19 PDT 2009,NO_QUERY,DeliciousLunch,"@Col_RFTL Well, we've been discussing Shrews d...","[col, rftl, well, discussing, shrews, deliveri..."
408695,0,2059463939,Sat Jun 06 16:59:20 PDT 2009,NO_QUERY,andretouch,Gay guys are mean to me or they're just bitch...,"[gay, guys, mean, bitches, whyyy]"
40959,0,1573921698,Tue Apr 21 03:29:12 PDT 2009,NO_QUERY,cittiecait,how come i never win anything: #lg15 #harpersg...,"[come, never, win, anything, lg, harpersglobe,..."


In [14]:
dataset_sample_with_bow = dataset_sample.copy()

In [16]:
dataset_sample_with_bow.to_csv('/Users/manuelaquino/Bootcamp/dataptmad1120/module-3/natural-language-processing/data/dataset_sample_with_bow.csv')

In [20]:
from nltk.probability import FreqDist

words_list = []

for row in tqdm(dataset_sample_with_bow["text_processed"]):
    for words in row:
        words_list.append(words)
        
fdist = FreqDist()
fdist = FreqDist(word for word in words_list)

top_words = [i for i in fdist.most_common(5000)]
print(top_words[:500])

100%|██████████| 20000/20000 [00:00<00:00, 355339.58it/s]


[('day', 1120), ('good', 1114), ('get', 996), ('like', 954), ('got', 887), ('go', 878), ('work', 854), ('today', 852), ('quot', 825), ('going', 781), ('love', 778), ('time', 737), ('lol', 736), ('back', 718), ('u', 689), ('know', 659), ('one', 654), ('im', 638), ('amp', 625), ('really', 612), ('still', 582), ('see', 561), ('think', 551), ('night', 538), ('home', 523), ('well', 519), ('oh', 513), ('na', 511), ('thanks', 506), ('want', 501), ('new', 495), ('much', 467), ('miss', 462), ('morning', 452), ('http', 439), ('last', 432), ('need', 421), ('tomorrow', 402), ('great', 398), ('hope', 390), ('twitter', 386), ('haha', 384), ('feel', 381), ('sad', 359), ('fun', 357), ('tonight', 353), ('right', 348), ('com', 346), ('wish', 340), ('sleep', 337), ('bad', 336), ('sorry', 335), ('happy', 327), ('come', 321), ('though', 321), ('would', 311), ('yeah', 305), ('week', 300), ('getting', 297), ('nice', 292), ('bed', 291), ('better', 289), ('gon', 289), ('make', 289), ('way', 284), ('watching', 

In [26]:
print(top_words[-200:])

[('thinker', 3), ('advil', 3), ('leopard', 3), ('becomes', 3), ('hmmmmm', 3), ('bebo', 3), ('storming', 3), ('remains', 3), ('defeat', 3), ('jake', 3), ('ward', 3), ('method', 3), ('rumour', 3), ('judging', 3), ('tees', 3), ('shaq', 3), ('sod', 3), ('geography', 3), ('cheesy', 3), ('fancy', 3), ('crisis', 3), ('playboy', 3), ('joint', 3), ('hmph', 3), ('boreddd', 3), ('hundred', 3), ('moral', 3), ('myweakness', 3), ('rod', 3), ('vines', 3), ('handing', 3), ('detroit', 3), ('entered', 3), ('yous', 3), ('nauseous', 3), ('aussiemcflyfan', 3), ('serving', 3), ('jamba', 3), ('quotes', 3), ('commission', 3), ('amazin', 3), ('animals', 3), ('bottom', 3), ('inspire', 3), ('wut', 3), ('gmail', 3), ('destroy', 3), ('jessemccartney', 3), ('whatsoever', 3), ('therefore', 3), ('thai', 3), ('nani', 3), ('smosh', 3), ('flips', 3), ('directly', 3), ('jab', 3), ('erica', 3), ('bahaha', 3), ('subtitles', 3), ('streamed', 3), ('devastated', 3), ('consuming', 3), ('folder', 3), ('git', 3), ('cv', 3), ('el

### Building Features

Now let's build the features. Using the top 5,000 words, create a 2-dimensional matrix to record whether each of those words is contained in each document (tweet). Then you also have an output column to indicate whether the sentiment in each tweet is positive. For example, assuming your bag of words has 5 items (`['one', 'two', 'three', 'four', 'five']`) out of 4 documents (`['A', 'B', 'C', 'D']`), your feature set is essentially:

| Doc | one | two | three | four | five | is_positive |
|---|---|---|---|---|---|---|
| A | True | False | False | True | False | True |
| B | False | False | False | True | True | False |
| C | False | True | False | False | False | True |
| D | True | False | False | False | True | False|

However, because the `nltk.NaiveBayesClassifier.train` class we will use in the next step does not work with Pandas dataframe, the structure of your feature set should be converted to the Python list looking like below:

```python
[
	({
		'one': True,
		'two': False,
		'three': False,
		'four': True,
		'five': False
	}, True),
	({
		'one': False,
		'two': False,
		'three': False,
		'four': True,
		'five': True
	}, False),
	({
		'one': False,
		'two': True,
		'three': False,
		'four': False,
		'five': False
	}, True),
	({
		'one': True,
		'two': False,
		'three': False,
		'four': False,
		'five': True
	}, False)
]
```

To help you in this step, watch the [following video](https://www.youtube.com/watch?v=-vVskDsHcVc) to learn how to build the feature set with Python and NLTK. The source code in this video can be found [here](https://pythonprogramming.net/words-as-features-nltk-tutorial/).

[![Building Features](building-features.jpg)](https://www.youtube.com/watch?v=-vVskDsHcVc)

In [ ]:
# your code here

### Building and Traininng Naive Bayes Model

In this step you will split your feature set into a training and a test set. Then you will create a Bayes classifier instance using `nltk.NaiveBayesClassifier.train` ([example](https://www.nltk.org/book/ch06.html)) to train with the training dataset.

After training the model, call `classifier.show_most_informative_features()` to inspect the most important features. The output will look like:

```
Most Informative Features
	    snow = True            False : True   =     34.3 : 1.0
	  easter = True            False : True   =     26.2 : 1.0
	 headach = True            False : True   =     20.9 : 1.0
	    argh = True            False : True   =     17.6 : 1.0
	unfortun = True            False : True   =     16.9 : 1.0
	    jona = True             True : False  =     16.2 : 1.0
	     ach = True            False : True   =     14.9 : 1.0
	     sad = True            False : True   =     13.0 : 1.0
	  parent = True            False : True   =     12.9 : 1.0
	  spring = True            False : True   =     12.7 : 1.0
```

The [following video](https://www.youtube.com/watch?v=rISOsUaTrO4) will help you complete this step. The source code in this video can be found [here](https://pythonprogramming.net/naive-bayes-classifier-nltk-tutorial/).

[![Building and Training NB](nb-model-building.jpg)](https://www.youtube.com/watch?v=rISOsUaTrO4)

In [1]:
# your code here

### Testing Naive Bayes Model

Now we'll test our classifier with the test dataset. This is done by calling `nltk.classify.accuracy(classifier, test)`.

As mentioned in one of the tutorial videos, a Naive Bayes model is considered OK if your accuracy score is over 0.6. If your accuracy score is over 0.7, you've done a great job!

In [ ]:
# your code here

## Bonus Question 1: Improve Model Performance

If you are still not exhausted so far and want to dig deeper, try to improve your classifier performance. There are many aspects you can dig into, for example:

* Improve stemming and lemmatization. Inspect your bag of words and the most important features. Are there any words you should furuther remove from analysis? You can append these words to further remove to the stop words list.

* Remember we only used the top 5,000 features to build model? Try using different numbers of top features. The bottom line is to use as few features as you can without compromising your model performance. The fewer features you select into your model, the faster your model is trained. Then you can use a larger sample size to improve your model accuracy score.

In [ ]:
# your code here

## Bonus Question 2: Machine Learning Pipeline

In a new Jupyter Notebook, combine all your codes into a function (or a class). Your new function will execute the complete machine learning pipeline job by receiving the dataset location and output the classifier. This will allow you to use your function to predict the sentiment of any tweet in real time. 

In [ ]:
# your code here

## Bonus Question 3: Apache Spark

If you have completed the Apache Spark advanced topic lab, what you can do is to migrate your pipeline from local to a Databricks Notebook. Share your notebook with your instructor and classmates to show off your achievements!

In [ ]:
# your code here